In [49]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from ipywidgets import interact

from ml_collections import ConfigDict
from models.ETD_KT_CM_JAX_Vectorised import ETD_KT_CM_JAX_Vectorised, ic, KS_params, KDV_params
from filters import resamplers
from filters.filter import ParticleFilter


In [ ]:
signal_params = ConfigDict(KDV_params)
signal_params.update(E=1)
ensemble_params = ConfigDict(KDV_params)
ensemble_params.update(E=10)
ensemble_params.update(sigma=0.001)
signal_model = ETD_KT_CM_JAX_Vectorised(signal_params)
ensemble_model = ETD_KT_CM_JAX_Vectorised(ensemble_params)

initial_signal = ic(signal_model.x, signal_params.E)
initial_ensemble = ic(ensemble_model.x, ensemble_params.E) 
#initial_ensemble += 0.01 * jax.random.normal(jax.random.PRNGKey(56), initial_ensemble.shape) # adding this is totally "unphysical"

available_resamplers = ", ".join(resamplers.keys())
print(available_resamplers)


multinomial, systematic, no_resampling, default


In [ ]:
resampling = 'no_resampling'

pf = ParticleFilter(
    n_particles = ensemble_params.E,
    n_steps = 100,
    n_dim = initial_signal.shape[-1],
    forward_model = ensemble_model,
    signal_model = signal_model,
    sigma = 0.1,
    seed = 11,
    resampling=resampling,
)

In [ ]:
final, all = pf.run(initial_ensemble, initial_signal, 10)

In [ ]:
print(type(all))
print(len(all))
print(all[0].shape) # particles
print(all[1].shape) # signal
#out shape is (n_total, n_particles, n_dim)

<class 'tuple'>
2
(10, 10, 256)
(10, 1, 256)


In [ ]:
print(initial_signal.shape)
signal = jnp.concatenate([initial_signal[None,...], all[1]], axis=0)
print(signal.shape)
particles = jnp.concatenate([initial_ensemble[None,...], all[0]], axis=0)
print(particles.shape)

(1, 256)
(11, 1, 256)
(11, 10, 256)


In [ ]:
def plot(da_step):
    plt.plot(signal_model.x, signal[da_step,0,:], color='red')
    plt.plot(signal_model.x, particles[da_step,:,:].T, color='grey', linewidth=0.5)
    plt.legend(['signal', 'particles'])
    plt.show()

interact(plot, da_step=(0, 11))

interactive(children=(IntSlider(value=5, description='da_step', max=11), Output()), _dom_classes=('widget-inte…

<function __main__.plot(da_step)>